# 비매너 댓글 식별

In [26]:
import pandas as pd
import numpy as np 

import os 
import json

import torch
import torch.nn as nn

from attrdict import AttrDict




from transformers import ( 
    BertConfig,
    ElectraConfig,
    ElectraConfig
)

from transformers import (
    BertTokenizer,  
    AutoTokenizer,
    ElectraTokenizer,
)

from transformers import (
    BertModel,
    AutoModel, 
    ElectraForSequenceClassification,
    BertForSequenceClassification
)


In [10]:
print("number of GPUs: ", torch.cuda.device_count())
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

use_cuda = torch.cuda.is_available()
print("Dose GPU exist : ", use_cuda)

DEVICE = torch.device("cuda" if use_cuda else 'cpu')
print('DEVICE : ', DEVICE)

number of GPUs:  1
Dose GPU exist :  True
DEVICE :  cuda


In [11]:
DEBUG = False

In [16]:
config_path = os.path.join('config2.json')

def set_config(config_path):
    if os.path.lexists(config_path):
        with open(config_path) as f:
            args = AttrDict(json.load(f))
            print("config file loaded")
            print(args.pretrained_model)
    else:
        assert False, "config json cannot be found.. please check the path again"
        
    return args

args = set_config(config_path)

os.makedirs(args.result_dir, exist_ok=True)
os.makedirs(args.config_dir, exist_ok=True)

config file loaded
beomi/kcbert-base


In [17]:
train_path = os.path.join(args.data_dir, 'train.csv')
train_df = pd.read_csv(train_path, encoding='UTF-8-SIG')

train_df.head()

,title,comment,bias,hate
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none


In [18]:
len(train_df)

8367

In [21]:
print('bias classes : ', train_df.bias.unique())
print('hate classes : ', train_df.hate.unique())

bias classes :  ['none' 'others' 'gender']
hate classes :  ['none' 'hate']


In [22]:
pd.crosstab(train_df.bias, train_df.hate, margins=True)

hate,hate,none,All
bias,,,
gender,1216,83,1299
none,2068,3422,5490
others,1437,141,1578
All,4721,3646,8367


In [23]:
test_path = os.path.join(args.data_dir, 'test.csv')

test_df = pd.read_csv(test_path)
test_df.head()

,ID,title,comment
0,0,"류현경♥︎박성훈, 공개연애 4년차 애정전선 이상無..""의지 많이 된다""[종합]",둘다 넘 좋다~행복하세요
1,1,"""현금 유도+1인 1라면?""…'골목식당' 백종원, 초심 잃은 도시락집에 '경악' [종합]",근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
2,2,"입대 D-11' 서은광의 슬픈 멜로디..비투비, 눈물의 첫 체조경기장[콘서트 종합]",누군데 얘네?
3,3,"아이콘택트' 리쌍 길, 3년 전 결혼설 부인한 이유 공개…""결혼,출산 숨겼다""","쑈 하지마라 짜식아!음주 1번은 실수, 2번은 고의, 3번은 인간쓰레기다.슬금슬금 ..."
4,4,"구하라, 안검하수 반박 해프닝...""당당하다""vs""그렇게까지"" 설전 [종합]",안검하수 가지고 있는 분께 희망을 주고 싶은건가요? 수술하면 이렇게 자연스러워진다고...


In [25]:
len(test_df)

511

In [34]:
# 두 라벨의 조합 세트
combinations = np.array(np.meshgrid(train_df.bias.unique(), train_df.hate.unique())).T.reshape(-1, 2)
print(combinations)

[['none' 'none']
 ['none' 'hate']
 ['others' 'none']
 ['others' 'hate']
 ['gender' 'none']
 ['gender' 'hate']]


In [48]:
# bias, hate 컬럼 종합
bias_hate = list(np.array([train_df['bias'].values, train_df['hate'].values]).T.reshape(-1, 2))

if DEBUG==True:
    print(bias_hate[:5])

In [49]:
labels = []
for i, arr in enumerate(bias_hate):
    for idx, elem in enumerate(combinations):
        if np.array_equal(elem, arr):
            labels.append(idx)
            
            
train_df['label'] = labels
train_df.head()

,title,comment,bias,hate,label
0,"""'미스터 션샤인' 변요한, 김태리와 같은 양복 입고 학당 방문! 이유는?""",김태리 정말 연기잘해 진짜,none,none,0
1,"""[SC현장]""""극사실주의 현실♥""""…'가장 보통의 연애' 김래원X공효진, 16년만...",공효진 발연기나이질생각이읍던데 왜계속주연일까,none,hate,1
2,"""손연재, 리듬체조 학원 선생님 """"하고 싶은 일 해서 행복하다""""""",누구처럼 돈만 밝히는 저급인생은 살아가지마시길~~ 행복은 머니순이 아니니깐 작은거에...,others,hate,3
3,"""'섹션TV' 김해숙 """"'허스토리' 촬영 후 우울증 얻었다""""""",일본 축구 져라,none,none,0
4,"""[단독] 임현주 아나운서 “‘노브라 챌린지’ 방송 덕에 낸 용기, 자연스런 논의의...",난 절대로 임현주 욕하는인간이랑은 안논다 @.@,none,none,0
